In [1]:
import numpy as np
import pandas as pd

import ipywidgets as widgets
from IPython.display import display, Math

In [2]:
import bokeh.plotting as bplt
from bokeh.models import Range1d, LabelSet, ColumnDataSource
from bokeh.models.glyphs import Line
from bokeh.io import output_notebook, push_notebook
output_notebook()

Loading BokehJS ...

# Random Walks: Section 1 - The Gambler's Ruin Problem

One example of a Discrete-Time Markov Chains that we saw in Section 7 was the “Gambler's Ruin Problem”:

* You start with an initial fortune of $i$ and play as follows:
* You make sequential bets and win 1 with probability $p$ or lose 1 with probability $1 - p$.
* You stop when you are broke ($0$) or rich (total fortune of $N$).

What is the probability you end up rich?

## Problem Formulation

To formulate the problem as a Markov chain on $\mathcal{S} = \{0, 1, 2, \dots, N\}$, let $X_n$ be your fortune after $n$ bets.  (So $\{X_n,\ n \geq 0\}$ is a stochastic process with $X_0 = i$, your initial fortune.)

On any bet, your fortune increases by 1 or decreases by 1 until you reach $0$ or $N$ (i.e., it is impossible for your fortune to change by any other amount).  We will model going broke or become rich as remaining in state $0$ or state $N$ (respectively) forever.  

Therefore, the transition probabilities are
$$\begin{aligned}
    p_{00} &= 1 \\
  p_{ij} &= \begin{cases}
    p, & j = i + 1\\
    1 - p, & j = i - 1
  \end{cases} \qquad i \in \{1,\dots,N-1\} \\
  p_{NN} &= 1
\end{aligned}$$

We can write it as a matrix, but most of the elements are 0 and the size of the state space is a parameter.

$$\mathbf{P} = \begin{pmatrix}
     1 & 0 & 0 & 0 & 0 & 0 & \dots & 0 \\
     1-p & 0 & p & 0 & 0 & 0 & \dots & 0\\
     0 & 1-p & 0 & p & 0 & 0 & \dots & 0 \\
     0 & 0 & 1-p & 0 & p & 0 & \dots & 0\\
     \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
     0 & \dots & 0 & 1-p & 0 & p & 0 & 0 \\
     0 & \dots & 0 & 0 & 1-p & 0 & p & 0 \\
     0 & \dots & 0 & 0 & 0 & 1-p & 0 & p \\
     0 & \dots & 0 & 0 & 0 & 0 & 0 & 1 \\
  \end{pmatrix}$$

The transition diagram is:

### Exercise

Specify the communication class(es) and identify states as transient or recurrent.

### Remarks

The dynamics of this Markov chain is similar to a “birth-death process” where each transition is either a birth (from $i$ to $i+1$) or a death (from $i$ to $i-1$) and no other transitions are allowed.  We will study birth-death processes in the context of queueing theory.

You now have a (slightly) new way to write the transition probabilities of a Markov chain: by specifying all the non-zero transition probabilities.  Remember, this is simply a new way of writing something you are already familiar with.

## Calculating Probabilities:

Let $q = 1 - p$ (the probability that you lose a bet).  This is just to have slightly more concise notation.

Let $P_i$ be the probability you end up rich starting with an initial fortune of $i$.  Conditioning on the outcome of the first bet:
$$P_i = p P_{i+1} + q P_{i-1}$$
since, given that you win the first bet, the probability you end up rich is $P_{i+1}$.

Since $p + q = 1$:
$$\begin{aligned}
    p P_i + q P_i &= p P_{i+1} + q P_{i-1} \\
    (P_{i+1} - P_i) &= \frac{q}{p} (P_i - P_{i-1})
  \end{aligned}$$
This gives us a *recursion relationship* and building from $P_1 - P_0 = P_1$ (since $P_0 = 0$):
$$\begin{aligned}
    P_1 - P_0 &= P_1 \\
    P_2 - P_1 &= \frac{q}{p} P_1 \\
    P_3 - P_2 &= \frac{q}{p} \left(P_2 - P_1\right) = \left(\frac{q}{p}\right)^2 P_1 \\
    &\ \vdots \\
    P_i - P_{i-1} &= \left(\frac{q}{p}\right)^{i-1} P_1 \\
    &\ \vdots \\
    P_N - P_{i-1} &= \left(\frac{q}{p}\right)^{N-1} P_1
  \end{aligned}$$
The left-hand side of each equation also gives us terms in a *telescoping series*, so:
$$\sum_{k=1}^i (P_j - P_{j-1}) = P_i = P_1 \sum_{k=1}^i \left(\frac{q}{p}\right)^{k-1} = P_1 \sum_{k=0}^{i-1} \left(\frac{q}{p}\right)^k$$
or
$$P_i = P_1 \sum_{k=0}^{i-1} \left(\frac{q}{p}\right)^k \qquad\text{for any } i \in \{1,\dots,N\}$$

### Case 1: $p = \frac{1}{2}$

If $p = \frac{1}{2} = q$ then $\frac{q}{p} = 1$ and so
$$P_i = P_1 \sum_{k=0}^{i-1} \left(\frac{q}{p}\right)^k = i P_1 \qquad i = 1, 2, \dots, N.$$

Since $P_N = 1$ (if you start in $N$ then you reach $N$ with probability 1), $P_N = 1 = N P_1$, so $P_1 = \frac{1}{N}$ and
$$P_i = \frac{i}{N} \qquad i \in \mathcal{S}$$

### Case 2: $p \neq \frac{1}{2}$

This follows the same approach, but looks a bit more complicated due to the summation
$$\sum_{k=0}^n x^k = \frac{1 - x^{n+1}}{1 - x} \qquad \text{for } x \neq 1$$
If $p \neq \frac{1}{2}$ then $p \neq q$ and $\frac{q}{p} \neq 1$ and so
$$P_i = P_1 \sum_{k=0}^{i-1} \left(\frac{q}{p}\right)^k = \frac{1 - (q/p)^i}{1 - (q/p)} P_1 \qquad i = 1, 2, \dots, N.$$

Since $P_N = 1$, we have
$$P_N = 1 = \frac{1 - (q/p)^N}{1 - (q/p)} P_1$$
and solving for $P_1$:
$$P_1 = \frac{1 - (q/p)}{1 - (q/p)^N}$$
and
$$P_i = \frac{1 - (q/p)^i}{1 - (q/p)^N} \qquad i \in \mathcal{S}$$

Combining the two cases, we can write:
$$P_i = \begin{cases}
    \displaystyle\frac{1 - (q/p)^i}{1 - (q/p)^N} & p \neq \frac{1}{2} \\
    \displaystyle\frac{i}{N} & p = \frac{1}{2}
  \end{cases}$$

## Becoming Infinitely Rich

This analysis applies for any $N$, so we can consider the limit $N \to \infty$ and ask what is the probability we become infinitely rich?

$$\lim_{N \to \infty} P_i = \begin{cases}
    1 - \left(\frac{q}{p}\right)^i & p > \frac{1}{2} \\
    0 & p \leq \frac{1}{2}
  \end{cases}$$
  
So, if the odds are in your favor ($p > 1/2$) then there is a non-zero probability that you will never go broke (but as long as $p < 1$, there remains a non-zero probability that you will eventually go broke).  However, if the odds are fair or against you ($p \leq 1/2$), you will eventually go broke with probability 1.

## Numerical Simulation and Sample Path Visualization

In [3]:
p = 0.55 # probability of winning one bet
N = 50 # stop playing when rich
X_0 = 10 # starting fortune

num_paths = 100
steps = 1000

In [4]:
# Simulate each step
delta = np.random.choice([-1,1], size=(steps,num_paths), p=[1-p, p])

# Simulate random walk
random_walk = X_0 + np.append(np.zeros((1, num_paths)), np.cumsum(delta, axis=0), axis=0)
sample_path = pd.DataFrame(random_walk, index=pd.Index(range(0,steps+1)), 
                           columns=[f'path {i}' for i in range(1,num_paths+1)])

# Account for absorbing states
sample_path = sample_path.mask(sample_path.cummin(axis=0).le(0), 0) # Absorb at 0
sample_path = sample_path.mask(sample_path.cummax(axis=0).ge(N), N) # Absorb at N (or comment out for N = infinity)

if p == 0.5:
    P_N = X_0/N
else:
    P_N = (1 - ((1-p)/p)**X_0)/(1-((1-p)/p)**N)

In [5]:
plt = bplt.figure(title="Gambler's Ruin")
plt.title.align = 'center'
plt.title.text_font_size = '25px'
source = ColumnDataSource(sample_path)
for col in sample_path.columns:
    color = "blue" if sample_path[col].max()==N else "red" if sample_path[col].min()==0 else "gray"
    plt.add_glyph(source, Line(x="index", y=col, line_color=color))
bplt.show(plt)
print(f'Prob(rich) = {P_N: .5f}')

Prob(rich) =  0.86561
